# desilike

## Código para especificar likelihoods de DESI

### Tutorial #2: bindings con otros códigos de inferencia cosmológica

# Metas
Al final de este tutorial, se habrá aprendido:
- Cómo escribir likelihoods en términos de parámetros comprimidos (por ejemplo, basándose en forecasts), opcionalmente con covarianza entre distintas mediciones
- Cómo escribir likelihoods de full shape, y emular su teoría
- Cómo correr inferencia dentro de desilike, o [Cobaya](https://github.com/CobayaSampler/cobaya), [CosmoSIS](https://github.com/joezuntz/cosmosis), y [MontePython](https://github.com/brinckmann/montepython_public).

Para ejemplos de la vida real, ver [mock Y1 cosmological inference](https://github.com/cosmodesi/desi-y1-kp7/tree/main/mock_y1).

# Likelihoods comprimidas

Primero enfoquémonos en likelihoods comprimidas = que sólo dependen en el modelo cosmológico (nuisance = bias, parámetros estocásticos y contratérminos ya marginalizados).

=> El sampleo de este tipo de likelihoods es típicamente sucifientemente rápido que no es necesario emularlo.

## Likelihood de BAO

Las likelihoods de BAO son construidas a partir de un "observable de BAO", que compara un data vector con una teoría, típicamente $\alpha_{\perp}$ y $\alpha_{\parallel}$ o $D_{M}/r_{d}$ y $D_{H}/r_{d}$.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import jax; jax.config.update('jax_platform_name', 'cpu')
%matplotlib inline

In [ ]:
import numpy as np

from desilike import utils, setup_logging
from desilike.likelihoods import ObservablesGaussianLikelihood
from desilike.observables.galaxy_clustering import BAOCompressionObservable

setup_logging()

# Por default elegimos la cosmología fiducial de DESI
observable1 = BAOCompressionObservable(data=[1., 1.],
                                       covariance=np.diag([0.01, 0.01]),
                                       quantities=['qpar', 'qper'],
                                       z=1.)
# Definimos la likelihood utilizando este observable
likelihood = ObservablesGaussianLikelihood(observable1)

### Recordatorio

In [ ]:
# Parámetros de la likelihood
print('Los parámetros a varias con esta likelihood son', likelihood.varied_params.names())
# Para evaluar la likelihood (obtener la logposterior)
print('logposterior es {:.3f}'.format(likelihood(Omega_m=0.29)))

¿Y por qué nos importan los "observables"?

... Porque podemos unirlos en una likelihood!

In [ ]:
# Queremos compartir el mismo cálculo cosmológico entre todos los observables
# Así que hagámoslo explícitamente
from desilike.theories import Cosmoprimo
cosmo = Cosmoprimo(fiducial='DESI')
# Definir los parámetros del calculador de Cosmoprimo
cosmo.init.params = {'Omega_m': {'prior': {'limits': [0.1, 0.9]},
                                 'ref': {'dist': 'norm', 'loc': 0.3, 'scale': 0.002},
                                 'latex': '\Omega_m'}}
# Reutilicemos el primer observable que definimos, sólo actualizando la cosmología
observable1.init.update(cosmo=cosmo)
# A ponernos fancy y en lugar de definir un observable en términos de DV_over_rd
# Proveamos un diccionaro a "data": el theory vector que será generado automáticamente
observable2 = BAOCompressionObservable(data={}, quantities=['DV_over_rd'], z=1.5, cosmo=cosmo)
# Unamos las dos observables, y proveamos la covarianza conjunta
likelihood = ObservablesGaussianLikelihood([observable1, observable2],
                                           covariance=np.diag([0.01, 0.01, 1.]))
print('likelihood is {:.4f}'.format(likelihood()))

También podemos sumar (log-) likelihoods!

In [ ]:
likelihood2 = likelihood + likelihood
likelihood(Omega_m=0.29)
likelihood2(Omega_m=0.29)
assert np.allclose(likelihood2.loglikelihood, 2. * likelihood.loglikelihood)

## Bindings
Para generar bindings a desilike, debemos empezar por escribir un llamable (~"función") que regrese la likelihood de desilike

In [ ]:
def BAOLikelihood(cosmo='external'):
    # cosmo = 'external' para decirle a desilike que la cosmología será porporcionada por
    # el código de inferencia externo
    observable1 = BAOCompressionObservable(data=[1., 1.], quantities=['qpar', 'qper'], z=0.5,
                                           cosmo=cosmo)
    observable2 = BAOCompressionObservable(data=[1.], quantities=['qiso'], z=1., cosmo=cosmo)
    likelihood = ObservablesGaussianLikelihood([observable1, observable2],
                                               covariance=np.diag([0.002, 0.002, 0.005]))
    return likelihood

### Cobaya
### Bindings 'Dinámicos'

Cobaya  está diseñado de manera que podemos proveer likelihoods definidos en la marcha (no necesariamente en un script), y llevar a cabo la inferencia directamente en Python.

In [ ]:
from desilike.bindings.cobaya import CobayaLikelihoodFactory

# CobayaBAOLikelihood es un objeto de Likelihood en Cobaya
CobayaBAOLikelihood = CobayaLikelihoodFactory(BAOLikelihood, params=True)

In [ ]:
from cosmoprimo.fiducial import DESI
cosmo = DESI()

# Nada mágico aquí, esto son puras cosas de Cobaya
params = {'Omega_m': {'prior': {'min': 0.1, 'max': 1.},
                      'ref': {'dist': 'norm', 'loc': 0.3, 'scale': 0.01},
                      'latex': '\Omega_{m}'},
          'omega_b': cosmo['omega_b'],
          'H0': cosmo['H0'],
          'A_s': cosmo['A_s'],
          'n_s': cosmo['n_s'],
          'tau_reio': cosmo['tau_reio']}

info = {'params': params,
        'likelihood': {'bao_likelihood': CobayaBAOLikelihood},
        'theory': {'classy': {'extra_args': {'N_ncdm': cosmo['N_ncdm'], 'N_ur': cosmo['N_ur']}}}}

from cobaya.model import get_model
model = get_model(info)
model.logposterior({'Omega_m': cosmo['Omega_m']})

In [ ]:
# Hora de correr MCMC!
info_sampler = {'mcmc': {'Rminus1_stop': 0.02}}
from cobaya.sampler import get_sampler
mcmc = get_sampler(info_sampler, model=model)
mcmc.run()

In [ ]:
from cobaya.sampler import get_sampler_name_and_class
get_sampler(info_sampler, model=model)

In [ ]:
from getdist.mcsamples import MCSamplesFromCobaya
samples_bao_cobaya = mcmc.samples(combined=True, skip_samples=0.5, to_getdist=True).copy(label='cobaya')
from getdist import plots
g = plots.get_subplot_plotter()
g.triangle_plot(samples_bao_cobaya, params=['Omega_m'], markers={'Omega_m': cosmo['Omega_m']})

### Interludio: Hagamos lo mismo desde desilike

In [ ]:
from desilike.samplers import MCMCSampler

from desilike.theories import Cosmoprimo
cosmo = Cosmoprimo(fiducial='DESI')
# Definir los parámetros del calculador de Cosmoprimo
cosmo.init.params = {'Omega_m': {'prior': {'limits': [0.1, 1.]},
                                 'ref': {'dist': 'norm', 'loc': 0.3, 'scale': 0.01},
                                 'latex': '\Omega_m'}}
sampler = MCMCSampler(BAOLikelihood(cosmo=cosmo), seed=42)
chains = sampler.run(check={'max_eigen_gr': 0.03, 'stable_over': 2}, check_every=40)
# usar help(chains[0]) para obtener información sobre todos los métodos disponibles!
samples_bao_desilike = chains[0].remove_burnin(0.5).to_getdist(label='desilike')

In [ ]:
from getdist import plots

g = plots.get_subplot_plotter()
g.triangle_plot([samples_bao_cobaya, samples_bao_desilike],
                 params=['Omega_m'], markers={'Omega_m': cosmo['Omega_m']})

### Bindings 'Estáticos' 

Otros códigos de inferencia (CosmoSIS, MontePython) típicamente requieren que la likelihood esté guardada en un archivo, de manera que puede ser importada por el código.
Para ilustrar esto, aún con Cobaya...

In [ ]:
!rm -rf _tests

In [ ]:
utils.mkdir('_tests')

In [ ]:
%%file _tests/bao_likelihood.py

dirname = '.'

def BAOLikelihood(cosmo='external'):
    import numpy as np
    from desilike.observables.galaxy_clustering import BAOCompressionObservable
    from desilike.likelihoods import ObservablesGaussianLikelihood
    # cosmo = 'external' para decirle a desilike que la cosmología será porporcionada externamente
    observable1 = BAOCompressionObservable(data=[1., 1.], quantities=['qpar', 'qper'], z=0.5, cosmo=cosmo)
    observable2 = BAOCompressionObservable(data=[1.], quantities=['qiso'], z=1., cosmo=cosmo)
    likelihood = ObservablesGaussianLikelihood([observable1, observable2],
                                               covariance=np.diag([0.002, 0.002, 0.005]))
    return likelihood

if __name__ == '__main__':
    from desilike.bindings import CobayaLikelihoodGenerator
    # Pudimos haber proporcionado una lista de Likelihoods, las cuales serán escritas todas a la vez
    CobayaLikelihoodGenerator(dirname=dirname)([BAOLikelihood], kw_like={'cosmo': 'external'})

Generemos los bindings estáticos llamando al script anterior de Python

In [ ]:
%%bash
cd _tests/
python bao_likelihood.py

Echemos un vistazo a los archivos generados:
- el módulo de Python que contiene la likelihood de Cobaya: ``bao_likelihood.py``
- importado en el archivo ``__init__.py``
- el archivo de configuración ``.yaml`` que contiene los parámetros nuisance (ninguno en este caso)

In [ ]:
!ls -la _tests/cobaya

In [ ]:
!cat _tests/cobaya/BAOLikelihood.yaml

In [ ]:
# %load _tests/cobaya/bao_likelihood.py
# NOTA: Este código ha sido automáticamente generado por desilike.bindings.cobaya.factory.CobayaLikelihoodGenerator
from desilike.bindings.cobaya.factory import CobayaLikelihoodFactory
from desilike.bindings.base import load_from_file
import os
current_directory = os.getcwd()

BAOLikelihood = load_from_file(f'{current_directory}/_tests/bao_likelihood.py', 'BAOLikelihood')
BAOLikelihood = CobayaLikelihoodFactory(BAOLikelihood, 'BAOLikelihood', {'cosmo': 'external'}, __name__)



Ahora escribamos el archivo de configuración para correr inferencia. Esto es Cobaya puro.

In [ ]:
%%file _tests/config_bao.yaml

theory:
  classy:
    extra_args:
      N_ncdm: 1
      N_ur: 2.0328

likelihood:
  bao_likelihood.BAOLikelihood:
      python_path: _tests/cobaya

params:
  Omega_m:
    prior:
      min: 0.1
      max: 1.
    ref:
      dist: norm
      loc: 0.3
      scale: 0.01
    latex: \Omega_{m}
  omega_b: 0.02237
  H0: 67.36
  As: 2.083e-09
  n_s: 0.9649
  tau_reio: 0.0544

sampler:
  mcmc:
    Rminus1_stop: 0.02

debug: False

output: _tests/chains_bao_cobaya/chain


A hacer sampling!

In [ ]:
!cobaya-run _tests/config_bao.yaml

In [ ]:
# Para cargar samples de Cobaya desde el disco
from getdist.mcsamples import loadMCSamples
samples_bao_cobaya = loadMCSamples('_tests/chains_bao_cobaya/chain', settings={'ignore_rows': 0.5}).copy(label='cobaya')

g = plots.get_subplot_plotter()
g.triangle_plot([samples_bao_cobaya, samples_bao_desilike],
                 params=['Omega_m'], markers={'Omega_m': cosmo['Omega_m']})

### CosmoSIS
Apliquemos los bindings estáticos a CosmoSIS.
La definición de BAOLikelihood es exactamente la misma, sólo necesitamos generar nuevos archivos con una simple llamada a CosmoSISLikelihoodGenerator.

In [ ]:
%%file _tests/bao_likelihood.py

dirname = '.'

# Igual que con Cobaya!
def BAOLikelihood(cosmo='external'):
    import numpy as np
    from desilike.observables.galaxy_clustering import BAOCompressionObservable
    from desilike.likelihoods import ObservablesGaussianLikelihood
    # cosmo = 'external' para decirle a desilike que la cosmología será porporcionada externamente
    observable1 = BAOCompressionObservable(data=[1., 1.], quantities=['qpar', 'qper'], z=0.5, cosmo=cosmo)
    observable2 = BAOCompressionObservable(data=[1.], quantities=['qiso'], z=1., cosmo=cosmo)
    likelihood = ObservablesGaussianLikelihood([observable1, observable2],
                                               covariance=np.diag([0.002, 0.002, 0.005]))
    return likelihood

if __name__ == '__main__':
    from desilike.bindings import CobayaLikelihoodGenerator, CosmoSISLikelihoodGenerator, MontePythonLikelihoodGenerator
    CobayaLikelihoodGenerator(dirname=dirname)([BAOLikelihood], kw_like={'cosmo': 'external'})
    # El único cambio!
    CosmoSISLikelihoodGenerator(dirname=dirname)([BAOLikelihood], kw_like={'cosmo': 'external'})
    # Generemos directamente los bindings para MontePython
    MontePythonLikelihoodGenerator(dirname=dirname)([BAOLikelihood], kw_like={'cosmo': 'external'})

Generemos los bindings estáticos llamando al script anterior de Python

In [ ]:
%%bash
cd _tests
python bao_likelihood.py

Echemos un vistazo a los archivos generados:
- El módulo de Python que contiene la likelihood de Cobaya: ``BAOLikelihood.py``
- El archivo ``*values.ini`` que contiene los valores / rangos de los parámetros nuisance (ninguno en este caso), a ser copiado-pegado en el archivo input ``*values.ini`` (ver más adelante)
- El archivo ``*priors.ini`` que contiene las priors opcionales de los parámetros nuisance, a ser copiado-pegado en el archivo input ``*priors.ini``

In [ ]:
ls -la _tests/cosmosis

Ahora escribamos el archivo de configuración para correr inferencia. Esto es CosmoSIS puro.

In [ ]:
%%file _tests/config_bao.ini

[DEFAULT]
fatal_errors = T

[runtime]
sampler = emcee

[output]
filename = _tests/chains_bao_cosmosis/chain.txt
format = text
verbosity = 0

[pipeline]
modules = consistency camb bao
values = _tests/values_bao.ini
likelihoods = BAOLikelihood  ; Nótese el nombre de la likelihood: el mismo que el archivo *.py
quiet = T
debug = F
timing = F

[consistency]
file = ${COSMOSIS_STD_DIR}/utility/consistency/consistency_interface.py

[camb]
file = ${COSMOSIS_STD_DIR}/boltzmann/camb/camb_interface.py
mode = background
feedback = 0
nz = 901

[bao]
file = _tests/cosmosis/BAOLikelihood.py

[emcee]
walkers = 6
samples = 600
nsteps = 20

El archivo ``*values.ini`` que contiene los valores y rangos de los parámetros

In [ ]:
%%file _tests/values_bao.ini

[cosmological_parameters]

; Este es el único parámetro siendo variado.
omega_m = 0.1 0.3 0.9
ombh2 = 0.02237
h0 = 0.6736
A_s = 2.083e-09
n_s = 0.9649
tau = 0.0544

mnu = 0.06
nnu = 3.046
num_massive_neutrinos = 1
omega_k = 0.0
w = -1.0
wa = 0.0

Hagamos el sampleo!

In [ ]:
!cosmosis _tests/config_bao.ini

In [ ]:
# Para cargar samples de CosmoSIS desde el disco
from cosmosis import Inifile
from cosmosis.output import input_from_options
from getdist import MCSamples

ini = Inifile('_tests/config_bao.ini')
options = dict(ini.items('output'))
options['filename'] = '_tests/chains_bao_cosmosis/chain.txt'
column_names, data = input_from_options(options)[:2]
#print(column_names)
data = data[0].T
data = data[..., data.shape[-1] // 2:]  # removiendo el burnin
samples_bao_cosmosis = MCSamples(samples=[data[0]], weights=None, loglikes=-data[-1],
                                 names=['Omega_m'], label='cosmosis')

g = plots.get_subplot_plotter()
g.triangle_plot([samples_bao_cobaya, samples_bao_desilike, samples_bao_cosmosis],
                 params=['Omega_m'], markers={'Omega_m': cosmo['Omega_m']})

### MontePython

MontePython no es un paquete de Python, así que no está instalado en el ambiente de cosmodesi.

Instalémoslo localmente! (Puede tomar algo de tiempo en descargar, debido al tamaño de los sets de datos).

In [ ]:
%%bash
cd _tests/
git clone https://github.com/brinckmann/montepython_public.git

Escribimos el archivo ``.conf`` que especifica el path al código de Boltzman 'Class' y a las likelihoods de Planck.

In [ ]:
%%file _tests/montepython_public/default.conf

import os
path['cosmo'] = os.getenv('CLASS_STD_DIR')
path['clik'] = os.path.join(os.getenv('PLANCK_SRC_DIR'), 'code', 'plc_3.0', 'plc-3.1')

Echemos un vistazo a los archivos previamente generados por los bindings estáticos:

- El paquete: ``BAOLikelihood``
- Con un archivo (obligatorio) ``*.data`` que especifica el nombre de la likelihood y los priors de los parámetros nuisance
- Junto al archivo ``*.param`` especificando rangos de parámetros, a ser copiado-pegado en el archivo input ``.param`` (ver más adelante)
- Con el arvhivo ``__init__.py`` que contiene la definición de la likelihood

Como MontePython lo requiere, copiamos todo esto a la carpeta ``montepython/likelihoods``.

In [ ]:
!ls -la _tests/montepython/BAOLikelihood
!cp -r _tests/montepython/BAOLikelihood _tests/montepython_public/montepython/likelihoods/

Ahora escribamos el archivo de configuración para correr inferencia. Esto es MontePython puro.

In [ ]:
%%file _tests/conf_bao.param

data.experiments = ['BAOLikelihood']

#------ Lista de parámetros-------
# data.parameters[nombre] = [media, min, max, 1-sigma, escala, rol]

# Lista de parámetros cosmológicos
data.parameters['Omega_m'] = [0.3, 0.1, 0.9, 0.1, 1., 'cosmo']
# Parámetros fijos
data.parameters['omega_b'] = [0.02237, 0.001, 0.1, 0., 1., 'cosmo']
data.parameters['H0'] = [67.36, 0.1, 0.9, 0., 1., 'cosmo']
data.parameters['A_s'] = [2.083e-09, 1e-09, 3e-09, 0., 1., 'cosmo']
data.parameters['n_s'] = [0.9649, 0.9, 1.0, 0., 1., 'cosmo']
data.parameters['tau_reio'] = [0.0544, 0.02, 0.1, 0., 1., 'cosmo']

# Argumentos cosmo
data.cosmo_arguments['k_pivot'] = 0.05
data.cosmo_arguments['N_ur'] = 2.0328
data.cosmo_arguments['N_ncdm'] = 1
data.cosmo_arguments['m_ncdm'] = 0.06
data.cosmo_arguments['T_ncdm'] = 0.71611

#------ Parámetros MCMC ----
data.N = 3000
data.write_step = 5

Hagamos el sampleo!

In [ ]:
!python _tests/montepython_public/montepython/MontePython.py run --conf _tests/montepython_public/default.conf -p _tests/conf_bao.param -o _tests/chains_bao_montepython

In [ ]:
# Para cargar samples de MontePython desde el disco
from datetime import date
from getdist.mcsamples import loadMCSamples
samples_bao_montepython = loadMCSamples('_tests/chains_bao_montepython/{}_3000_'.format(date.today()),
                                    settings={'ignore_rows': 0.5}).copy(label='montepython')

g = plots.get_subplot_plotter()
g.triangle_plot([samples_bao_cobaya, samples_bao_desilike, samples_bao_cosmosis, samples_bao_montepython],
                 params=['Omega_m'], markers={'Omega_m': cosmo['Omega_m']})

## Interludio: forecasts
Hasta ahora hemos usado como input matrices de covarianza completamente ad-hoc.
A pesar de esto, es posible utilizar desilike para producir forecasts.

In [ ]:
from desilike.observables.galaxy_clustering import CutskyFootprint
from desilike.theories.galaxy_clustering import BAOPowerSpectrumTemplate, SimpleBAOWigglesTracerPowerSpectrumMultipoles
from desilike.likelihoods.galaxy_clustering import SNWeightedPowerSpectrumLikelihood
from desilike import Fisher

cosmo = DESI()

# Objeto que contiene el área y n(z) en unidades de (Mpc/h)^(-3)
footprint = CutskyFootprint(area=14000., zrange=np.linspace(0.8, 1.2, 10), nbar=np.full(10, 1e-4), cosmo=cosmo)
z = footprint.zavg

fo = cosmo.get_fourier()
s, s0 = fo.sigma8_z(z, of='delta_cb'), fo.sigma8_z(0., of='delta_cb')
b1 = 0.8 / (s / s0)  # Prescripción para el bias lineal
r = 0.5  # Factor de reconstrucción
sigmaper = 9.4 * (s / 0.9)
f = fo.sigma8_z(z, of='theta_cb') / s
params = {'b1': b1, 'sigmapar': r * (1. + f) * sigmaper, 'sigmaper': r * sigmaper}  # Parámetros del modelo fiducial
covariance_params = {'b1': b1, 'sigmapar': 0., 'sigmaper': 0.}  # Parámetros de la covarianza fiducial (modelo Kaiser simple)
template = BAOPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qparqper')
theory = SimpleBAOWigglesTracerPowerSpectrumMultipoles(template=template) # Este modelo de BAO sólo traslada los wiggles
for param in theory.init.params.select(basename='al*'):
    param.update(value=0., fixed=True)  # Fijamos los parámetros broadband (sólo los wiggles se trasladan)

# Para klim=(0.01, 0.5), solo usamos la información del pico de BAO en el espectro de potencias
likelihood = SNWeightedPowerSpectrumLikelihood(theories=theory, data=params, covariance=covariance_params,
                                               footprints=footprint, klim=(0.01, 0.5))
fisher = Fisher(likelihood)  # Inicializamos Fisher
fisher_bao = fisher(**params).view(params=['qpar', 'qper'])  # Calculamos la predicción Fisher con los parámetros fiduciales

In [ ]:
print(fisher_bao.to_stats(tablefmt='pretty'))

In [ ]:
quantities = ['qpar', 'qper']
# Se puede pasar directamente Fisher a BAOCompressionObservable
observable = BAOCompressionObservable(data=fisher_bao, covariance=fisher_bao,
                                      quantities=quantities, z=z)
# O...
observable = BAOCompressionObservable(data=fisher_bao.mean(quantities),
                                      covariance=fisher_bao.covariance(quantities),
                                      quantities=quantities, z=z)

Este observable puede ser pasado a una likelihood, justo como previamente, para realizar inferencia cosmológica.

## Likelihood de ShapeFit
Las likelihoods comprimidas de ShapeFit son similares a las likelihoods comprimidas de BAO.

In [ ]:
from desilike.observables.galaxy_clustering import ShapeFitCompressionObservable

observable = ShapeFitCompressionObservable(data=[1., 1., 1., 0.], covariance=np.diag([0.01, 0.01, 0.01, 0.01]),
                                           quantities=['qpar', 'qper', 'df', 'dm'], z=1.)
# Definamos la likelihood a partir de este observable
likelihood = ObservablesGaussianLikelihood(observable)

Este observable puede ser pasado a una likelihood, justo como previamente, para realizar inferencia cosmológica.

# Likelihoods completas

Escribamos likelihoods completas, con parámetros nuisance (bias, estocásticos y contratérminos que no han sido marginalizados).

In [ ]:
%%file _tests/fs_likelihood.py
dirname = '_tests'

def FSLikelihood(cosmo='external'):
    from desilike.theories.galaxy_clustering import DirectPowerSpectrumTemplate, KaiserTracerPowerSpectrumMultipoles, LPTVelocileptorsTracerPowerSpectrumMultipoles
    from desilike.observables.galaxy_clustering import BoxFootprint, ObservablesCovarianceMatrix, TracerPowerSpectrumMultipolesObservable
    from desilike.likelihoods import ObservablesGaussianLikelihood
    # Definamos el template = espectro de potencias lineal
    template = DirectPowerSpectrumTemplate(z=1.)
    # Con el propósito de ahorrar tiempo computacional,
    # consideremos un modelo lineal Kaiser simple
    theory = KaiserTracerPowerSpectrumMultipoles(template=template)
    b1 = 0.5
    footprint = BoxFootprint(volume=5e9, nbar=1e-4)  # caja con volumen de 5 (Gpc/h)^3 y densidad de 1e-4 (h/Mpc)^3
    observable = TracerPowerSpectrumMultipolesObservable(\
                 data={'b1': b1},  # path de los datos, archivo *pypower*, arreglo, o diccionario de parámetros
                 covariance=None,  # path de los mocks, arreglo (matriz de covarianzas), o None
                 klim={0: [0.01, 0.2, 0.01], 2: [0.01, 0.2, 0.01]},  # k-limits, entre 0.01 y 0.2 h/Mpc con pasos de 0.005 h/Mpc
                 theory=theory)  # teoría definida previamente
    covariance = ObservablesCovarianceMatrix(observables=[observable], footprints=[footprint])
    cov = covariance(b1=b1)  # evaluar matriz de covarianzas en este parámetro
    likelihood = ObservablesGaussianLikelihood(observables=observable, covariance=cov)
    observable.init.update(data=observable.flatdata)  # fijar el data vector
    template.init.update(cosmo=cosmo)  # pasemos la cosmología
    return likelihood

if __name__ == '__main__':
    from desilike.bindings import CobayaLikelihoodGenerator, CosmoSISLikelihoodGenerator, MontePythonLikelihoodGenerator
    CobayaLikelihoodGenerator(dirname=dirname)([FSLikelihood], kw_like={'cosmo': 'external'})
    CosmoSISLikelihoodGenerator(dirname=dirname)([FSLikelihood], kw_like={'cosmo': 'external'})
    MontePythonLikelihoodGenerator(dirname=dirname)([FSLikelihood], kw_like={'cosmo': 'external'})

Generemos los bindings estáticos llamando al script anterior en Python

In [ ]:
!python _tests/fs_likelihood.py

In [ ]:
!ls -la _tests/cobaya

In [ ]:
!ls -la _tests/cosmosis

In [ ]:
!ls -la _tests/montepython

Y a pesar de todo, la likelihood anterior tomará un tiempo apreciable en evaluarse, especialmente para un modelo EFT con 1-loop.
Emulemos la teoría.

In [ ]:
# %load _tests/fs_likelihood.py
dirname = '_tests'

def FSLikelihood(cosmo='external'):
    from desilike.theories.galaxy_clustering import DirectPowerSpectrumTemplate, KaiserTracerPowerSpectrumMultipoles, LPTVelocileptorsTracerPowerSpectrumMultipoles
    from desilike.observables.galaxy_clustering import BoxFootprint, ObservablesCovarianceMatrix, TracerPowerSpectrumMultipolesObservable
    from desilike.likelihoods import ObservablesGaussianLikelihood
    # Definamos el template = espectro de potencias lineal
    template = DirectPowerSpectrumTemplate(z=1.)
    # Con el propósito de ahorrar tiempo computacional,
    # consideremos un modelo lineal Kaiser simple
    theory = KaiserTracerPowerSpectrumMultipoles(template=template)
    b1 = 1.5
    footprint = BoxFootprint(volume=5e9, nbar=1e-4)  # caja con un volumen de 5 (Gpc/h)^3 y densidad de 1e-4 (h/Mpc)^3
    observable = TracerPowerSpectrumMultipolesObservable(\
                 data={'b1': b1},  # path de los datos, archivo *pypower*, arreglo, o diccionario de parámetros
                 covariance=None,  # path de los mocks, arreglo (matriz de covarianzas), o None
                 klim={0: [0.01, 0.2, 0.01], 2: [0.01, 0.2, 0.01]},  # k-limits, entre 0.01 y 0.2 h/Mpc con pasos de 0.005 h/Mpc
                 theory=theory)  # teoría definida previamente
    covariance = ObservablesCovarianceMatrix(observables=[observable], footprints=[footprint])
    cov = covariance(b1=b1)  # evaluar matriz de covarianzas en este parámetro
    likelihood = ObservablesGaussianLikelihood(observables=observable, covariance=cov)
    observable.init.update(data=observable.flatdata)  # fijar el data vector
    template.init.update(cosmo=cosmo)  # pasemos la cosmología
    return likelihood

In [ ]:
likelihood = FSLikelihood(cosmo=None)
theory = likelihood.observables[0].wmatrix.theory

from desilike.emulators import Emulator, TaylorEmulatorEngine, EmulatedCalculator

emulator = Emulator(theory.pt,
                    engine=TaylorEmulatorEngine(order={'*': 1}))
emulator.set_samples()
emulator.fit()  # definir expansión de Taylor

# El emulador se puede guardar con:
emulator.save('_tests/emulator.npy')
np.save('_tests/data.npy', likelihood.flatdata)
np.save('_tests/covariance.npy', likelihood.covariance)

theory.init.update(pt=emulator)

Ahora podemos escribir nuestra likelihood, usando teoría de perturbación emulada!

In [ ]:
%%file _tests/fs_likelihood.py
dirname = '_tests'

def FSLikelihood():
    import os
    import numpy as np
    from desilike.theories.galaxy_clustering import DirectPowerSpectrumTemplate, KaiserTracerPowerSpectrumMultipoles
    from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable
    from desilike.likelihoods import ObservablesGaussianLikelihood
    from desilike.emulators import EmulatedCalculator
    # Definamos el template
    template = DirectPowerSpectrumTemplate(z=1.)
    # Con el propósito de ahorrar tiempo computacional,
    # consideremos un modelo lineal Kaiser simple
    theory = KaiserTracerPowerSpectrumMultipoles(template=template,
                                                 pt=EmulatedCalculator.load(os.path.join(dirname, 'emulator.npy')))
    observable = TracerPowerSpectrumMultipolesObservable(\
                 data={'b1': b1},  # path de los datos, archivo *pypower*, arreglo, o diccionario de parámetros
                 klim={0: [0.01, 0.2, 0.01], 2: [0.01, 0.2, 0.01]},  # k-limits, entre 0.01 y 0.2 h/Mpc con pasos de 0.005 h/Mpc
                 theory=theory,
                 covariance=np.load(os.path.join(dirname, 'covariance.npy')))
    likelihood = ObservablesGaussianLikelihood(observables=observable)
    likelihood.all_params['b1'].update(ref=dict(limits=[1., 2.]))
    likelihood.all_params['sn0'].update(derived='.auto')
    return likelihood

if __name__ == '__main__':
    from desilike.bindings import CobayaLikelihoodGenerator, CosmoSISLikelihoodGenerator, MontePythonLikelihoodGenerator
    CobayaLikelihoodGenerator(dirname=dirname)(FSLikelihood, kw_like={})
    CosmoSISLikelihoodGenerator(dirname=dirname)(FSLikelihood, kw_like={})
    MontePythonLikelihoodGenerator(dirname=dirname)(FSLikelihood, kw_like={})

Generemos los bindings estáticos llamando al script anterior en Python

In [ ]:
!python _tests/fs_likelihood.py

### Cobaya

In [ ]:
%%file _tests/config_bao_fs.yaml

theory:
  classy:
    extra_args:
      N_ncdm: 1
      N_ur: 2.0328

likelihood:
  bao_likelihood.BAOLikelihood:
      python_path: _tests/cobaya
  fs_likelihood.FSLikelihood:
      python_path: _tests/cobaya

params:
  Omega_m:
    prior:
      min: 0.1
      max: 1.
    ref:
      dist: norm
      loc: 0.3
      scale: 0.01
    latex: \Omega_{m}
  omega_b: 0.02237
  H0: 67.36
  As: 2.083e-09
  n_s: 0.9649
  tau_reio: 0.0544

sampler:
  mcmc:
    Rminus1_stop: 0.05

debug: False

output: _tests/chains_bao_fs_cobaya/chain

Let's sample!

In [ ]:
!cobaya-run _tests/config_bao_fs.yaml

In [ ]:
from getdist.mcsamples import loadMCSamples
samples_bao_fs_cobaya = loadMCSamples('_tests/chains_bao_fs_cobaya/chain',
                                      settings={'ignore_rows': 0.5}).copy(label='cobaya')

### CosmoSIS

In [ ]:
%%file _tests/config_bao_fs.ini

[DEFAULT]
fatal_errors = T

[runtime]
sampler = emcee

[output]
filename = _tests/chains_bao_fs_cosmosis/chain.txt
format = text
verbosity = 0

[pipeline]
modules = consistency camb bao fs
values = _tests/values_bao_fs.ini
likelihoods = BAOLikelihood FSLikelihood  ; nótese el nombre de la likelihood: el mismo que el archivo *.py
quiet = T
debug = F
timing = F

[consistency]
file = ${COSMOSIS_STD_DIR}/utility/consistency/consistency_interface.py

[camb]
file = ${COSMOSIS_STD_DIR}/boltzmann/camb/camb_interface.py
mode = background
feedback = 0
; Necesitamo espaciado bastante fino en el redshift, ya que estamos usando supernovae
; bajamos a z bajo donde las cosas se ponen muy sensibles
nz = 901

[bao]
file = _tests/cosmosis/BAOLikelihood.py

[fs]
file = _tests/cosmosis/FSLikelihood.py

[emcee]
walkers = 10
samples = 800
nsteps = 20

En este caso la likelihood tiene parámetros nuisance, a ser copiados en el archivo input ``*values.ini``

In [ ]:
!cat _tests/cosmosis/FSLikelihood_values.ini

In [ ]:
%%file _tests/values_bao_fs.ini

[desi]
sigmapar = 0.0
sigmaper = 0.0
b1 = 0.0 1.5 4.0

[cosmological_parameters]
; Este es el único parámetro a ser variado.
omega_m = 0.1 0.3 0.9
ombh2 = 0.02237
h0 = 0.6736
A_s = 2.083e-09
n_s = 0.9649
tau = 0.0544

mnu = 0.06
nnu = 3.046
num_massive_neutrinos = 1
omega_k = 0.0
w = -1.0
wa = 0.0

Hagamos el sampleo!

In [ ]:
!cosmosis _tests/config_bao_fs.ini

In [ ]:
# Para cargar samples de CosmoSIS
from cosmosis import Inifile
from cosmosis.output import input_from_options
from getdist import MCSamples

ini = Inifile('_tests/config_bao_fs.ini')
options = dict(ini.items('output'))
options['filename'] = '_tests/chains_bao_fs_cosmosis/chain.txt'
column_names, data = input_from_options(options)[:2]
#print(column_names)
data = data[0].T
data = data[:2, data.shape[-1] // 2::10]  # remover burning
samples_bao_fs_cosmosis = MCSamples(samples=np.column_stack(data), weights=None, loglikes=-data[-1],
                                    names=['b1', 'Omega_m'], label='cosmosis')

### MontePython

In [ ]:
!ls -la _tests/montepython/FSLikelihood
!cp -r _tests/montepython/FSLikelihood _tests/montepython_public/montepython/likelihoods/

In this case the likelihood has nuisance parameters, to be copied in the input ``*.param`` file.

In [ ]:
!cat _tests/montepython/FSLikelihood/FSLikelihood.param

In [ ]:
%%file _tests/conf_bao_fs.param

data.experiments = ['BAOLikelihood', 'FSLikelihood']

# Lista de parámetros cosmológicos
data.parameters['Omega_m'] = [0.3, 0.1, 0.9, 0.1, 1., 'cosmo']
# Parámetros fijos
data.parameters['omega_b'] = [0.02237, 0.001, 0.1, 0., 1., 'cosmo']
data.parameters['H0'] = [67.36, 0.1, 0.9, 0., 1., 'cosmo']
data.parameters['A_s'] = [2.083e-09, 1e-09, 3e-09, 0., 1., 'cosmo']
data.parameters['n_s'] = [0.9649, 0.9, 1.0, 0., 1., 'cosmo']
data.parameters['tau_reio'] = [0.0544, 0.02, 0.1, 0., 1., 'cosmo']

# Lista de parámetros Nuisance
data.parameters['sigmapar'] = [0.0, 0.0, 10.0, 0.0, 1.0, 'nuisance']
data.parameters['sigmaper'] = [0.0, 0.0, 10.0, 0.0, 1.0, 'nuisance']
data.parameters['b1'] = [1.5, 0.0, 4.0, 0.28867513459481287, 1.0, 'nuisance']

# Argumentos Cosmo
data.cosmo_arguments['k_pivot'] = 0.05
# El modelo base incluye dos neutrinos
# sin masa y un neutrino masivo con m=0.06eV.
# Los ajustes siguientes garantizan que Neff=3.046
# y m/omega = 93.14 eV
data.cosmo_arguments['N_ur'] = 2.0328
data.cosmo_arguments['N_ncdm'] = 1
data.cosmo_arguments['m_ncdm'] = 0.06
data.cosmo_arguments['T_ncdm'] = 0.71611

#------ parámetros MCMC ----
# Número de pasos a tomar, por default (sobreescrito por el comando -N)
data.N = 9000
# Número de pasos aceptados antes de escribir al archivo de las chains. 
# Mientras más grande el número, se accesa menos al disco,
# pero realistamente esto no consume mucho tiempo
data.write_step = 5

Hagamos el sampleo!

In [ ]:
!python _tests/montepython_public/montepython/MontePython.py run --conf _tests/montepython_public/default.conf -p _tests/conf_bao_fs.param -o _tests/chains_bao_fs_montepython

In [ ]:
# Para cargar samples de MontePython
from datetime import date
from getdist.mcsamples import loadMCSamples
samples_bao_fs_montepython = loadMCSamples('_tests/chains_bao_fs_montepython/{}_9000_'.format(date.today()),
                                           settings={'ignore_rows': 0.5}).copy(label='montepython')

In [ ]:
g = plots.get_subplot_plotter()
g.triangle_plot([samples_bao_fs_cobaya, samples_bao_fs_cosmosis, samples_bao_fs_montepython],
                 params=['Omega_m', 'b1'], markers={'Omega_m': cosmo['Omega_m'], 'b1': 1.5})

# Recapitulación

- likelihoods comprimidas o full-shape son definidas solamente una vez, dentro de desilike
- bindings para Cobaya, CosmoSIS y MontePython son generados con un simple llamado de función
- para Cobaya, la likelihood generada se puede importar directamente
- para CosmoSIS y MontePython, la descripción de los parámetros nuisance debe ser copiada en el archivo de configuración usado como input